In [35]:
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]
P4 = [2, 4, 3, 1]
IP = [2, 6, 3, 1, 4, 8, 5, 7]
IP_inv = [4, 1, 3, 5, 7, 2, 8, 6]
EP = [4, 1, 2, 3, 2, 3, 4, 1]
# S-boxes
S0 = [
 [1, 0, 3, 2],
 [3, 2, 1, 0],
 [0, 2, 1, 3],
 [3, 1, 3, 2]
]
S1 = [
 [0, 1, 2, 3],
 [2, 0, 1, 3],
 [3, 0, 1, 0],
 [2, 1, 0, 3]
]
# Permutation function
def permutate(table, block):
 return [block[x - 1] for x in table]
# Left shift function
def left_shift(block, n):
 return block[n:] + block[:n]
# Key generation function
def generate_keys(key):
 key = permutate(P10, key)
 left_half, right_half = key[:5], key[5:]
 left_half = left_shift(left_half, 1)
 right_half = left_shift(right_half, 1)
 k1 = permutate(P8, left_half + right_half)
 
 left_half = left_shift(left_half, 2)
 right_half = left_shift(right_half, 2)
 k2 = permutate(P8, left_half + right_half)
 
 return k1, k2
# XOR function
def xor(bits1, bits2):
 return [b1 ^ b2 for b1, b2 in zip(bits1, bits2)]
# S-box lookup
def sbox_lookup(box, bits):
 row = (bits[0] << 1) | bits[3]
 col = (bits[1] << 1) | bits[2]
 return [(box[row][col] >> 1) & 1, box[row][col] & 1]
# Function F in S-DES
def f(bits, key):
 bits = permutate(EP, bits)
 bits = xor(bits, key)
 
 left_bits = sbox_lookup(S0, bits[:4])
 right_bits = sbox_lookup(S1, bits[4:])
 
 return permutate(P4, left_bits + right_bits)
# S-DES encryption function
def sdes_encrypt(plain_text, key):
 k1, k2 = generate_keys(key)
 bits = permutate(IP, plain_text)
 left_bits, right_bits = bits[:4], bits[4:]
 
 result = xor(left_bits, f(right_bits, k1))
 result = right_bits + result
 
 left_bits, right_bits = result[:4], result[4:]
 result = xor(left_bits, f(right_bits, k2))
 
 return permutate(IP_inv, result + right_bits)
# S-DES decryption function
def sdes_decrypt(cipher_text, key):
 k1, k2 = generate_keys(key)
 bits = permutate(IP, cipher_text)
 left_bits, right_bits = bits[:4], bits[4:]
 
 result = xor(left_bits, f(right_bits, k2))
 result = right_bits + result
 
 left_bits, right_bits = result[:4], result[4:]
 result = xor(left_bits, f(right_bits, k1))
 
 return permutate(IP_inv, result + right_bits)
# Input and Output Example
key = [1, 0, 1, 0, 0, 0, 0, 0, 1, 0] # 10-bit key
plain_text = [1, 0, 1, 0, 0, 0, 1, 0] # 8-bit plain text
cipher_text = sdes_encrypt(plain_text, key)
decrypted_text = sdes_decrypt(cipher_text, key)
print(f"Plain Text: {plain_text}")
print(f"Cipher Text: {cipher_text}")
print(f"Decrypted Text: {decrypted_text}")

Plain Text: [1, 0, 1, 0, 0, 0, 1, 0]
Cipher Text: [0, 1, 1, 0, 0, 0, 1, 1]
Decrypted Text: [1, 0, 1, 0, 0, 0, 1, 0]


In [34]:
!pip install pycryptodome